In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import optimizers

import urllib
import cv2
import os
import numpy as np

Using TensorFlow backend.


In [2]:
# Building the structure of VGGNet(with 16 layers)
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2), dim_ordering="th"))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(11, activation='softmax'))
    
    if weights_path:
        model.load_weights(weights_path, by_name=True)
        
    #Remove the last two layers to get the 4096D activations
    model.layers.pop()
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []

    return model
    

In [3]:
# Loading the pretrained weights for VGGNet
model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((2, 2), strides=(2, 2), data_format="channels_first")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  '` call to the Keras 2 API: ' + signature)
/home/blank/anaconda2/lib/python2.7/site-packages/keras/legacy/

In [4]:
# Compiling the model with stochastic gradient descent optimizer
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# 'sparse_categorical_crossentropy' loss can be used as alternative to one hot vectors if number of classes are large
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
"""
dataset is divided into training set(with 8000 images) and validation set(with 1338 images)
Underlying code is executed for both training and validation set, but only validation conditions are mentioned as
the code was first executed on training data and then on validation data
"""
num_of_images = 1338
# creating an array of size 4096 as the last layer of VGGNet provides a vector of length 4096
valid_image_arr_4096 = np.ndarray((num_of_images, 4096))
# array for storing labels corresponding images
valid_label_arr = np.ndarray((num_of_images, ), dtype=int)

# loading images from the directory
categories = os.listdir("valid/")
# dictionary for computing labels
label_dict = dict()

# label index
label = 1
# count for label array and image array
count = 0

# iterating over images stored in [train] and [valid] folders 
for img_type in categories:
    # initialize label vector and dictionary
    label_dict[img_type] = label
        
    path = "./valid/" + img_type
    for image in os.listdir(path):
        valid_label_arr[count] = label
        
        # read image as a vector and adjust its dimension according to the input dimensions of VGGNet
        img_path = path + '/' + image
        img = cv2.resize(cv2.imread(img_path),(224,224))
        img = img.transpose((2,0,1))
        img = np.expand_dims(img, axis=0)
        # getting vector of dimension (1, 4096)
        x = model.predict(img)
        valid_image_arr_4096[count] = x[0]
        count += 1
        # break when sufficient images are loaded
        if count == num_of_images: break
    # increase index for each different label
    label += 1
    if count == num_of_images: break

In [10]:
# save all the computed array and dictionaries
np.save('valid_image_arr_4096', valid_image_arr_4096)
np.save('valid_label_arr', valid_label_arr)
np.save('label_dict', label_dict)